# Домашнее задание 11


В этом задании вам предстоит построить архитектуру автоэнкодера, который будет сжимать и восстанавливать лица людей.

## Задание: Создание модели Vanilla AE

Ваша задача — реализовать класс атоэнкодера для данных.

Обратите внимание на то, сколько карт активации должно быть в последнем слое сети.


Вы можете варьировать количества блоков/слоев и устройства блоков. Архитектура блока (conv -> batch norm-> relu -> maxpool) подойдет.

In [ ]:
import torch
import torch.nn as nn
import numpy as np

def encoder_block(in_channels, out_channels, kernel_size=3, padding=1):
    """
    Encoder block:
    Conv -> BatchNorm -> ReLU -> Conv -> BatchNorm -> ReLU -> MaxPool
    Reduces spatial resolution by 2 while increasing channels.
    """
    block = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    return block

def decoder_block(in_channels, out_channels, kernel_size=3, padding=1):
    """
    Decoder block:
    TransposedConv (upsample) -> BatchNorm -> ReLU -> Conv -> BatchNorm -> ReLU
    Doubles spatial resolution while reducing channels.
    """
    block = nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )
    return block

class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()

        # Encoder: 64x64 -> 32x32 -> 16x16 -> 8x8
        self.encoder = nn.Sequential(
            encoder_block(3, 64),      # -> (64, 32, 32)
            encoder_block(64, 128),    # -> (128, 16, 16)
            encoder_block(128, 256)    # -> (256, 8, 8)
        )

        # Decoder: 8x8 -> 16x16 -> 32x32 -> 64x64
        self.decoder = nn.Sequential(
            decoder_block(256, 128),   # -> (128, 16, 16)
            decoder_block(128, 64),    # -> (64, 32, 32)
            nn.ConvTranspose2d(64, 3, kernel_size=2, stride=2),  # -> (3, 64, 64)
            nn.Sigmoid()  # keep outputs in [0, 1]
        )

    def forward(self, x):
        latent = self.encoder(x)       # downsampling
        reconstruction = self.decoder(latent)  # upsampling
        return reconstruction


Ячейка ниже проверяет, что модель работает правильно:

In [ ]:
# проверка, что у модели есть обучаемые слои
model = Autoencoder()
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
assert num_params > 10

# проверка, что модель собрана верно
random_tensor = torch.Tensor(np.random.random((32, 3, 64, 64)))
model = Autoencoder()
out = model(random_tensor)
assert out.shape == (32, 3, 64, 64), "неверный размер выхода модели"

# проверка, что у модели можно отцепить декодер и использовать его как
# отдельную модель
# если здесь возникла ошибка, убедитесь, что в вашей сети нет skip connection
random_tensor = torch.Tensor(np.random.random((32, 3, 64, 64)))
model = Autoencoder()
latent_shape = model.encoder(random_tensor).shape
latent = torch.Tensor(np.random.random(latent_shape))
out = model.decoder(latent)

### Сдача задания

Если обе ячейки отработали без ошибок, можно сдавать задание в первую задачу на Я.Контесте. Для этого нужно скопировать класс Autoencoder в нужное место в submission_template10.py и отправить submission_template10.py в Я.Контест.